In [ ]:
import pandas as pd
from binance import Client
from datetime import datetime
import numpy as np
from sklearn.linear_model import LogisticRegression


In [ ]:
api_key = "5zGsMawDMZTpG6O4PufKiWDA6D6sjWpi4oAnl8OrzMpAJBCl9gHzQEzJWSCfG8qS"
api_secret = "dfwUSdQl6sKaHCNMn31JmZAvmZDWEzzyZVf3INEoY1pFBIVBGmHAMVaKk1hQIVot"
client = Client(api_key, api_secret)

In [82]:
data = pd.DataFrame(client.get_historical_klines("BTCBUSD", Client.KLINE_INTERVAL_1DAY, str(datetime(2020,1,1)), str(datetime.utcnow())))

#format columns name
data.columns = ['datetime', 'open', 'high', 'low', 'close', 'volume','close_time', 'qav', 'num_trades','taker_base_vol', 'taker_quote_vol', 'ignore']
data.index = [datetime.fromtimestamp(x/1000.0) for x in data.datetime]
df = data.astype(float)

In [83]:
# "RETURNS" :  the percentage change from previous day (close column considered)
df['returns'] = df.close.pct_change() # relative price change on the current time frame 

In [84]:
# "DIRECTION" : translate returns into binary results
df['direction'] = np.where(df.returns > 0,1,0) # 1 if returns > 0 and 0 otherwise

In [85]:
df.direction.value_counts() # check the balance between both sides

1    598
0    564
Name: direction, dtype: int64

In [86]:
# "LAGGED RETURNS" : add the n_lags previous returns next to the current one in a dataframe

def lagReturns(df, lags): 
    for i in range(1,lags+1):
        df[f'lag_{i}'] = df['returns'].shift(i)
    return [f'lag_{i}' for i in range(1, lags+1)]

In [87]:
features = lagReturns(df,2) # add 2 lagged returns
df.dropna(inplace=True) # drop NaN rows

In [88]:
x = df[features] # indep var - dataframe of n lags

In [89]:
y = df['direction'] # dependent var - dataframe of the directions ( 0 or 1 )

In [90]:
# Setting up the Machine Learning Model
model = LogisticRegression(class_weight='balanced')
model.fit(x,y) # train the model

LogisticRegression(class_weight='balanced')

In [91]:
# Make the predictions from the logistic regression model
df['prediction_LR'] = model.predict(x)

# The model will show 1 if we should be in the market and 0 if we shouldn't

In [92]:
# This strategy is getting a return if the prediction return 1 because we are in the market
# If the strategy predicted 0, we are not in the market so no returns

df['strategy_returns'] = df['prediction_LR'] * df.returns

,datetime,open,high,low,close,volume,close_time,qav,num_trades,taker_base_vol,taker_quote_vol,ignore,returns,direction,lag_1,lag_2,prediction_LR,strategy_returns
2020-01-04 01:00:00,1.578096e+12,7330.91,7388.00,7263.55,7346.20,299.060756,1.578182e+12,2.189479e+06,12455.0,124.078576,9.083768e+05,0.0,0.002319,1,0.055270,-0.032223,0,0.000000
2020-01-05 01:00:00,1.578182e+12,7346.20,7486.66,7315.00,7351.42,363.286518,1.578269e+12,2.695304e+06,12115.0,189.832412,1.408108e+06,0.0,0.000711,1,0.002319,0.055270,0,0.000000
2020-01-06 01:00:00,1.578269e+12,7349.84,7794.98,7343.29,7755.10,556.563567,1.578355e+12,4.212965e+06,17522.0,265.831971,2.012334e+06,0.0,0.054912,1,0.000711,0.002319,1,0.054912
2020-01-07 01:00:00,1.578355e+12,7757.48,8204.71,7730.00,8154.74,1019.907075,1.578442e+12,8.101274e+06,25599.0,444.632899,3.533457e+06,0.0,0.051533,1,0.054912,0.000711,0,0.000000
2020-01-08 01:00:00,1.578442e+12,8150.58,8459.66,7825.00,8047.67,1249.764389,1.578528e+12,1.025313e+07,32990.0,573.413295,4.702983e+06,0.0,-0.013130,0,0.051533,0.054912,0,-0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-03-03 01:00:00,1.677802e+12,23467.76,23478.79,21979.28,22355.86,136356.690140,1.677888e+12,3.055138e+09,2673686.0,67792.457160,1.518851e+09,0.0,-0.047420,0,-0.007029,0.021224,1,-0.047420
2023-03-04 01:00:00,1.677888e+12,22356.50,22411.17,22159.35,22347.17,37168.830800,1.677974e+12,8.296041e+08,1149792.0,18125.643560,4.045672e+08,0.0,-0.000389,0,-0.047420,-0.007029,1,-0.000389
2023-03-05 01:00:00,1.677974e+12,22347.12,22661.16,22188.00,22431.94,54931.469880,1.678061e+12,1.232224e+09,1464792.0,27719.726990,6.218096e+08,0.0,0.003793,1,-0.000389,-0.047420,1,0.003793
2023-03-06 01:00:00,1.678061e+12,22431.37,22600.00,22264.00,22408.20,62828.896640,1.678147e+12,1.409018e+09,1738365.0,31401.929140,7.042287e+08,0.0,-0.001058,0,0.003793,-0.000389,0,-0.000000


In [93]:
# Evaluate the performance
# We need to compare the returns from the strategy and the returns from Bitcoin itself
(df[['strategy_returns','returns']] + 1).cumprod()

,strategy_returns,returns
2020-01-04 01:00:00,1.000000,1.002319
2020-01-05 01:00:00,1.000000,1.003032
2020-01-06 01:00:00,1.054912,1.058110
2020-01-07 01:00:00,1.054912,1.112637
2020-01-08 01:00:00,1.054912,1.098028
...,...,...
2023-03-03 01:00:00,5.294530,3.050246
2023-03-04 01:00:00,5.292472,3.049060
2023-03-05 01:00:00,5.312548,3.060626
2023-03-06 01:00:00,5.312548,3.057387


In [94]:
# RESULTS : great performance 
# BUT - The model is trained on the whole dataset - overfitted model

# To do a realistic backtest, we need to train the model with a prior dataset than the one for backtest
# And check if the model is still outpermorming Bitcoin on newer data

from sklearn.model_selection import train_test_split

In [95]:
# Set indep & dependent variables to train the model AND to backtest
# NB : test_size = 0.3 ==> 30% data used to backtest & 70% ==> train the model
# NB : shuffle = False because we have a time serie - order matter

x_train, x_backtest, y_train, y_backtest = train_test_split(x,y,test_size=0.3, shuffle=False)

In [96]:
model.fit(x_train,y_train) # train the model with corresponding dataset

LogisticRegression(class_weight='balanced')

In [97]:
x_backtest['prediction_LR'] = model.predict(x_backtest)

In [98]:
x_backtest['returns'] = df.returns[x_backtest.index[0]:]
x_backtest['strategy_returns'] = x_backtest['prediction_LR'] * x_backtest.returns


In [99]:
(x_backtest[['strategy_returns','returns']] + 1).cumprod()

,strategy_returns,returns
2022-03-25 00:00:00,1.000000,1.007328
2022-03-26 00:00:00,1.000000,1.011737
2022-03-27 00:00:00,1.000000,1.064287
2022-03-28 00:00:00,1.000000,1.071147
2022-03-29 00:00:00,1.000000,1.078123
...,...,...
2023-03-03 01:00:00,0.700331,0.507933
2023-03-04 01:00:00,0.700059,0.507735
2023-03-05 01:00:00,0.702714,0.509661
2023-03-06 01:00:00,0.702714,0.509122
